In [4]:
import pandas as pd
from scrapingStates import get_state
import re

In [5]:
df = pd.read_csv("C:/Users/shado/Desktop/Programación(python,mySQL)/python-notebooks/datos/jobs_df.csv")
df.head(5)

,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,size,founded,type,industry,sector,revenue
0,"Data Scientist, Data Science & Modeling",CardWorks,"South Jordan, UT","$134,452 /yr (est.)",3.1,Merrick Bank employees share in our mission to...,501 to 1000 Employees,1987,Company - Private,Banking & Lending,Financial Services,$50 to $100 million (USD)
1,Data Science Engineer,Moorecroft Systems,Remote,$52.50 /hr (est.),NaN,Senior Data Science Engineer\nMoorecroft is lo...,51 to 200 Employees,Company - Public,Unknown / Non-Applicable,NaN,NaN,NaN
2,Student Employee - Data Science,Electric Power Research Institute,"Charlotte, NC","$82,980 /yr (est.)",5.0,Job Title:\nStudent Employee - Data Science\nL...,Unknown,Company - Private,Unknown / Non-Applicable,NaN,NaN,NaN
3,Data Science Engineer,Dataquestcorp,"Branchburg, NJ","$75,000 /yr (est.)",NaN,We are providing free training and placement o...,NaN,NaN,NaN,NaN,NaN,NaN
4,Junior Data Scientist,Augray,"Brookfield, WI","$57,500 /yr (est.)",4.3,"Brookfield, WI, USA\nSummary:\nHiring Fresher ...",51 to 200 Employees,2014,Subsidiary or Business Segment,Information Technology Support Services,Information Technology,Unknown / Non-Applicable


In [6]:
# hacemos un pequeño vistaso a las columnas del dataframe para tener una idea de que datos contiene
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Job Title        500 non-null    object 
 1   Company Name     500 non-null    object 
 2   Location         500 non-null    object 
 3   Salary Estimate  500 non-null    object 
 4   Rating           402 non-null    float64
 5   Job Description  500 non-null    object 
 6   size             306 non-null    object 
 7   founded          306 non-null    object 
 8   type             306 non-null    object 
 9   industry         262 non-null    object 
 10  sector           262 non-null    object 
 11  revenue          223 non-null    object 
dtypes: float64(1), object(11)
memory usage: 47.0+ KB


In [7]:
# diferenciamos los salarios ganados por año, por hora y por mes
df['yearly'] = df['Salary Estimate'].apply(lambda x: 2 if '/yr' in x else (1 if '/hr' in x else 0))
df['yearly'].value_counts()

2    449
1     43
0      8
Name: yearly, dtype: int64

In [8]:
# creamos una función lambda para separar los valores númericos de las letras
f_lambda = lambda x: x.split(' /yr')[0] if 'yr' in x else (x.split(' /hr')[0] if '/hr' in x else x.split(' /mo')[0])
df['salary'] = df['Salary Estimate'].apply(f_lambda).str.translate({ord('$'): None, ord(','): None}) # remplazamos los caracteres por un valor Nulo

In [9]:
# eliminate the hourly and month estimate jobs
# Eliminamos los trabajos que tienen un salario estimado en hora y mes
df = df[(df['yearly'] == 2)]

In [10]:
# transformamos la columna salary a númerica
df.salary = df.salary.astype(int)
df.salary.value_counts()

90375     130
80000      16
131513     15
180000     11
110000     10
         ... 
85503       1
75756       1
55466       1
120000      1
134425      1
Name: salary, Length: 111, dtype: int64

In [11]:
# LLamamos a una función de scraping que nos devolvera un diccionario con los estados y sus abreviaciones
states = get_state(4) # el parametro que recibe son los segundos que duerme la función
states.values()

dict_values(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Carolina del Norte', 'Carolina del Sur', 'Colorado', 'Connecticut', 'Dakota del Norte', 'Dakota del Sur', 'Delaware', 'Florida', 'Georgia', 'Hawái', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Luisiana', 'Maine', 'Maryland', 'Massachusetts', 'Míchigan', 'Minnesota', 'Misisipi', 'Misuri', 'Montana', 'Nebraska', 'Nevada', 'Nueva Jersey', 'Nueva York', 'Nuevo Hampshire', 'Nuevo México', 'Ohio', 'Oklahoma', 'Oregón', 'Pensilvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Virginia Occidental', 'Washington', 'Wisconsin', 'Wyoming'])

In [12]:
# Obtenemos los estados en base a su abreviación
f_discreta = lambda x: x if "," not in x else (states[x.split(",")[1]] if x.split(",")[1] in states.keys() else states[x.split(", ")[1]])
df['job_state'] = df.Location.apply(f_discreta)
df.job_state.unique()

array(['Utah', 'Carolina del Norte', 'Nueva Jersey', 'Wisconsin',
       'Remote', 'Nueva York', 'Georgia', 'Colorado', 'Illinois', 'Iowa',
       'Texas', 'Maryland', 'Ohio', 'California', 'Carolina del Sur',
       'Míchigan', 'Tennessee', 'Massachusetts', 'Virginia',
       'Pensilvania', 'Connecticut', 'Washington', 'Misuri'], dtype=object)

In [13]:
# se crea una lista de skill para buscar en la descripción del trabajo
# con ayuda del for se crea una columna por cada skill buscada y se le asigna 1 si es requerida y 0 si no lo es 
data_science_skills_list = ["python","java", "javascript", "sql", "excel", "spark", "aws", "azure" ]
for skill in data_science_skills_list:
    df[skill] =  df['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)
    print(f"skill : {skill}", )
    print(df[skill].value_counts())
    print("________________________________")


skill : python
0    395
1     54
Name: python, dtype: int64
________________________________
skill : java
0    449
Name: java, dtype: int64
________________________________
skill : javascript
0    449
Name: javascript, dtype: int64
________________________________
skill : sql
0    415
1     34
Name: sql, dtype: int64
________________________________
skill : excel
0    430
1     19
Name: excel, dtype: int64
________________________________
skill : spark
0    436
1     13
Name: spark, dtype: int64
________________________________
skill : aws
0    442
1      7
Name: aws, dtype: int64
________________________________
skill : azure
0    446
1      3
Name: azure, dtype: int64
________________________________


In [14]:
def puesto(title_job):
    if re.findall("(^| |[,-])data(|$)", title_job, re.I):
        if re.findall("(^| |,)scientist(|$)", title_job, re.I):
            return "data scientist"
        elif re.findall("(^| |,)learning(|$)", title_job, re.I):
            return "mle"
        elif re.findall("(^| |,)engineer(|$)", title_job, re.I):
            return "data engineer"
        elif re.findall("(^| |,)analyst(|$)", title_job, re.I):
            return "data analyst"
        elif re.findall("(^| |,)manager(|$)", title_job, re.I):
            return "manager"
        elif re.findall("(^| |,)analytics(|$)", title_job, re.I):
            return "data analytics"
        elif re.findall("(^| |,)science(|$)", title_job, re.I):
            return "data science"
        else:
            return "na"
    elif re.findall("(^| |,)analyst(|$)", title_job, re.I):
            return "analyst"
    else:
        return "na"

In [15]:
# obtenemos los puestos requeridos en los trabajos y los guardamos en una columna llamada job_simp
df['job_simp'] = df['Job Title'].apply(puesto)
df['job_simp'].value_counts()

data engineer     198
data scientist    157
data science       49
data analyst       21
data analytics     10
manager             7
mle                 4
analyst             2
na                  1
Name: job_simp, dtype: int64

In [16]:
def seniority(title):
    if re.findall("(^| |[\\,-/)])(sr|senior)(|$)", title, re.I):
        return "sr"
    elif re.findall("(^| |[\\,-/)])(jr|junior)(|$)", title, re.I):
        return "jr"
    else:
        return "mid"

In [17]:
# obtenemos el nivel de experiencia esperado para el trabajo del titulo y lo asignamos a una nueva columna
df['seniority'] = df['Job Title'].apply(seniority)
df.seniority.value_counts()

mid    415
jr      31
sr       3
Name: seniority, dtype: int64

In [18]:
# obtenemos el número de caracteres utilizados en la descripción del empleo y lo asignamos a una nueva columna
df['desc len'] = df['Job Description'].apply(lambda x: len(x))
df['desc len']

0      936
2      639
3      282
4      643
5      570
      ... 
495    804
496    804
497    804
498    804
499    804
Name: desc len, Length: 449, dtype: int64

In [19]:
df.head()

,Job Title,Company Name,Location,Salary Estimate,Rating,Job Description,size,founded,type,industry,...,java,javascript,sql,excel,spark,aws,azure,job_simp,seniority,desc len
0,"Data Scientist, Data Science & Modeling",CardWorks,"South Jordan, UT","$134,452 /yr (est.)",3.1,Merrick Bank employees share in our mission to...,501 to 1000 Employees,1987,Company - Private,Banking & Lending,...,0,0,0,0,0,0,0,data scientist,mid,936
2,Student Employee - Data Science,Electric Power Research Institute,"Charlotte, NC","$82,980 /yr (est.)",5.0,Job Title:\nStudent Employee - Data Science\nL...,Unknown,Company - Private,Unknown / Non-Applicable,NaN,...,0,0,0,0,0,0,0,data science,mid,639
3,Data Science Engineer,Dataquestcorp,"Branchburg, NJ","$75,000 /yr (est.)",NaN,We are providing free training and placement o...,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,data engineer,mid,282
4,Junior Data Scientist,Augray,"Brookfield, WI","$57,500 /yr (est.)",4.3,"Brookfield, WI, USA\nSummary:\nHiring Fresher ...",51 to 200 Employees,2014,Subsidiary or Business Segment,Information Technology Support Services,...,0,0,0,0,0,0,0,data scientist,jr,643
5,Data science instructor,MIM,Remote,"$131,513 /yr (est.)",NaN,ABOUT MIM\nMIM is a New York based institution...,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,data science,mid,570


In [20]:
df.describe()

,Rating,yearly,salary,python,java,javascript,sql,excel,spark,aws,azure,desc len
count,377.000000,449.0,449.000000,449.000000,449.0,449.0,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000
mean,3.965782,2.0,104640.505568,0.120267,0.0,0.0,0.075724,0.042316,0.028953,0.015590,0.006682,902.002227
std,0.577662,0.0,35678.082938,0.325637,0.0,0.0,0.264851,0.201534,0.167862,0.124022,0.081558,662.747769
min,1.000000,2.0,46789.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,118.000000
25%,3.900000,2.0,90375.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,643.000000
50%,4.000000,2.0,90375.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,804.000000
75%,4.200000,2.0,113281.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,882.000000
max,5.000000,2.0,275000.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,7444.000000


In [21]:
df.to_csv("C:/Users/shado/Desktop/Programación(python,mySQL)/python-notebooks/datos/salary_data_cleaned.csv", index=False)